In [ ]:
import json
from datetime import datetime
from tqdm import tqdm
from shroom_classifier_v13 import ShroomClassifier

In [ ]:
MODEL_NAME = "gpt-3.5-turbo"

MODEL_AGNOSTIC_FILENAME = 'reference/val.model-agnostic.json'

MODEL_AWARE_FILENAME = 'reference/val.model-aware.json'

ABLATION_STUDY_FILENAME = 'ablation_study_model-agnostic_2.json'

In [ ]:
def run_experiment(config):
    start = datetime.utcnow().isoformat()
    cp = ShroomClassifier(config["model_name"], config["temperature"], config["examples_per_class"])
    predictions = []
    data = json.load(open(config["dataset_filename"], 'r'))
    for dp in tqdm(data):
        prediction = cp.classify(
            dp, 
            N=config["samples"], 
            task_defined=config["task_defined"], 
            role_defined=config["role_defined"], 
            hallucination_defined=config["hallucination_defined"], 
            examples=config["examples"]
        )
        predictions.append(prediction)
    return { 
        "start": start, 
        "end": datetime.utcnow().isoformat(), 
        "config": config, 
        "data": data, 
        "predictions": predictions 
    }

In [ ]:
ablation_settings = [
    { "temperature": 0.5, "samples": 15, "examples_per_class": 3, "examples": True, "task_defined": True, "role_defined": True, "hallucination_defined": True },
    { "temperature": 0.5, "samples": 15, "examples_per_class": 3, "examples": False, "task_defined": True, "role_defined": True, "hallucination_defined": True },
    { "temperature": 0.5, "samples": 15, "examples_per_class": 3, "examples": False, "task_defined": False, "role_defined": True, "hallucination_defined": True },
    { "temperature": 0.5, "samples": 15, "examples_per_class": 3, "examples": False, "task_defined": False, "role_defined": False, "hallucination_defined": True },
    { "temperature": 0.5, "samples": 15, "examples_per_class": 3, "examples": False, "task_defined": False, "role_defined": False, "hallucination_defined": False },
]

In [ ]:
ablation_configs = []
for setting in ablation_settings:
    for dataset in [ MODEL_AGNOSTIC_FILENAME ]: # MODEL_AWARE_FILENAME ]: 
        for run in range(3):
            ablation_configs.append(
                {
                    "run": run,
                    "model_name": MODEL_NAME,
                    "dataset_filename": dataset,
                    "temperature": setting["temperature"],
                    "examples_per_class": setting["examples_per_class"],
                    "samples": setting["samples"],
                    "task_defined": setting["task_defined"],
                    "role_defined": setting["role_defined"],
                    "hallucination_defined": setting["hallucination_defined"],
                    "examples": setting["examples"]
                }
            )
len(ablation_configs)

In [ ]:
runs = []
for config in ablation_configs:
    print(config)
    runs.append(run_experiment(config))
    json.dump(runs, open(ABLATION_STUDY_FILENAME, 'w+'))